In [4]:
from model_funcs import dataset
from model_funcs import train_and_test
from model_funcs import model
import os
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np

test = dataset.SpectraDataset('test')
loader_test = DataLoader(test, batch_size= 1, pin_memory= True, shuffle=True)


In [5]:
model = torch.load('models/convolutionalnet.pth')

/tmp/ipykernel_34026/774777495.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('models/convolutionalnet.pth')


In [20]:
for X , env in loader_test:
    print('oi')
    break

oi


In [23]:
env

tensor([[330.1102,   0.7113]])

In [7]:
X , env = test[0]

In [8]:
X

tensor([[[-110.0000, -109.9990, -109.9997,  ..., -109.9993, -109.9986,
          -109.9972],
         [-109.9999, -109.9992, -109.9997,  ..., -109.9994, -109.9989,
          -109.9977],
         [-109.9999, -109.9996, -109.9998,  ..., -109.9997, -109.9994,
          -109.9988],
         ...,
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000],
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000],
         [-110.0000, -110.0000, -110.0000,  ..., -110.0000, -110.0000,
          -110.0000]]])

In [9]:
env

tensor([0.6981, 0.1448])

In [10]:
model.eval()
with torch.no_grad():
    pred = model(X.unsqueeze(0))

In [11]:
pred

tensor([[1., 1.]])

In [13]:
env[0]*400, env[1]

(tensor(279.2565), tensor(0.1448))

In [10]:
env[0] = env[0]/400

In [11]:
env

tensor([0.6981, 0.1448])